In [1]:
#User Behavior Data from Taobao for Recommendation（下载自阿里云天池）
#1.概述
#UserBehavior是阿里巴巴提供的一个淘宝用户行为数据集，用于隐式反馈推荐问题的研究。

#2.介绍
#文件名称	说明	包含特征
#UserBehavior.csv	包含所有的用户行为数据	用户ID，商品ID，商品类目ID，行为类型，时间戳
#UserBehavior.csv
#本数据集包含了2017年11月25日至2017年12月3日之间，有行为的约一百万随机用户的所有行为（行为包括点击、购买、加购、喜欢）。数据集的组织形式和MovieLens-20M类似，即数据集的每一行表示一条用户行为，由用户ID、商品ID、商品类目ID、行为类型和时间戳组成，并以逗号分隔。关于数据集中每一列的详细描述如下：

#列名称	说明
#用户ID	整数类型，序列化后的用户ID
#商品ID	整数类型，序列化后的商品ID
#商品类目ID	整数类型，序列化后的商品所属类目ID
#行为类型	字符串，枚举类型，包括('pv', 'buy', 'cart', 'fav')
#时间戳	行为发生的时间戳
#注意到，用户行为类型共有四种，它们分别是

#行为类型	说明
#pv	商品详情页pv，等价于点击
#buy	商品购买
#cart	将商品加入购物车
#fav	收藏商品

#关于数据集大小的一些说明如下
#维度	数量
#用户数量	987,994
#商品数量	4,162,024
#用户数量	987,994
#商品类目数量	9,439
#所有行为数量	100,150,807

In [2]:
import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

In [3]:
userbehavior = pd.read_csv('UserBehavior.csv')

In [4]:
userbehavior.shape

(100150806, 5)

In [6]:
#数据量过大，随机抽样2000000条
sample_data=userbehavior.sample(n=2000000)

In [7]:
sample_data.head()

,1,2268318,2520377,pv,1511544070
53129121,68396,926802,1080785,pv,1511714942
8906520,486815,1733438,4758477,pv,1511881864
60129248,1007183,4707318,149192,cart,1512228369
40153872,1010430,4681706,2253970,pv,1511839728
55236994,782032,775632,4731191,pv,1511758955


In [8]:
#缺少列名,补充
sample_data.columns=['User_Id','Sku_Id','Category_Id','Type','Timestamp']

In [9]:
sample_data.isnull().sum()

User_Id        0
Sku_Id         0
Category_Id    0
Type           0
Timestamp      0
dtype: int64

In [10]:
sample_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000000 entries, 53129121 to 22255036
Data columns (total 5 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   User_Id      int64 
 1   Sku_Id       int64 
 2   Category_Id  int64 
 3   Type         object
 4   Timestamp    int64 
dtypes: int64(4), object(1)
memory usage: 91.6+ MB


In [11]:
sample_data[sample_data["Timestamp"]<0].shape

(8, 5)

In [12]:
#有8行时间戳为负值的异常行，去掉
sample_data=sample_data[sample_data["Timestamp"]>0]

In [13]:
sample_data["Type"].value_counts()

pv      1791130
cart     110534
fav       57588
buy       40740
Name: Type, dtype: int64

In [14]:
sample_data.loc[:,'Timestamp']=sample_data['Timestamp'].apply(lambda x: time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(x)))
sample_data.loc[:,'Date']=sample_data['Timestamp'].apply(lambda x:x.split(' ')[0])
sample_data.loc[:,'Time']=sample_data['Timestamp'].apply(lambda x:x.split(' ')[1])

In [15]:
print(sample_data["Date"].value_counts())

2017-12-02    275039
2017-12-03    273841
2017-12-01    216303
2017-11-26    212857
2017-11-30    209164
               ...  
2015-11-29         1
2017-12-05         1
2010-01-04         1
2025-10-27         1
2017-11-16         1
Name: Date, Length: 65, dtype: int64


In [19]:
#存在异常日期，只取七天
userdata=sample_data[(sample_data["Date"]>='2017-11-27')&(sample_data["Date"]<='2017-12-03')]

In [20]:
print(userdata["Date"].value_counts())

2017-12-02    275039
2017-12-03    273841
2017-12-01    216303
2017-11-30    209164
2017-11-29    204473
2017-11-27    201244
2017-11-28    197389
Name: Date, dtype: int64


In [21]:
maindata=pd.DataFrame({"用户数量":len(set(list(userdata["User_Id"]))),"商品数量":len(set(list(userdata["Sku_Id"]))),"商品类目数量":len(set(list(userdata["Category_Id"]))),"统计天数":len(set(list(userdata["Date"])))},index=['1'])
maindata

,用户数量,商品数量,商品类目数量,统计天数
1,646387,644917,6894,7


In [22]:
userdata.to_csv('userdata.csv',sep=',')